In [1]:
import gymnasium as gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from collections import namedtuple, deque
from itertools import count

from gridworld import GridWorldEnv
import pygame

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import os
import numpy as np
import imageio.v2 as imageio

from tqdm.notebook import tqdm

prev_driver = os.environ.get("SDL_VIDEODRIVER")
os.environ["SDL_VIDEODRIVER"] = "dummy"

c:\Users\isaac\dev\DQN-Vis\venv\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [2]:
# where to save artifacts
OUT_DIR = "viz_out"
FRAME_DIR = os.path.join(OUT_DIR, "frames")
HEATMAP_DIR = os.path.join(OUT_DIR, "heatmaps")
ACTION_MAP_DIR = os.path.join(OUT_DIR, "action_maps")
GIF_DIR = os.path.join(OUT_DIR, "gifs")

os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(HEATMAP_DIR, exist_ok=True)
os.makedirs(ACTION_MAP_DIR, exist_ok=True)
os.makedirs(FRAME_DIR, exist_ok=True)
os.makedirs(GIF_DIR, exist_ok=True)

# (adjust if your env uses a different action ordering)
# 0:Up, 1:Right, 2:Down, 3:Left is a common GridWorld convention
ACTION_TO_VEC = {
    0: (0, -1),   # up
    1: (1,  0),   # right
    2: (0,  1),   # down
    3: (-1, 0),   # left
}

STEP_FRAME_EVERY = 1
GIF_FPS = 6  # adjust playback speed

In [3]:
env = GridWorldEnv(grid_size=6, render_mode="rgb_array", cell_size=72)

In [4]:
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if GPU is to be used
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)
print(f"device: {device}")

device: cpu


In [5]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [6]:
class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [7]:
# BATCH_SIZE is the number of transitions sampled from the replay buffer
# GAMMA is the discount factor as mentioned in the previous section
# EPS_START is the starting value of epsilon
# EPS_END is the final value of epsilon
# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay
# TAU is the update rate of the target network
# LR is the learning rate of the ``AdamW`` optimizer

BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.01
EPS_DECAY = 2500
TAU = 0.005
LR = 3e-4


# Get number of actions from gym action space
n_actions = env.action_space.n
# Get the number of state observations
state, info = env.reset()
n_observations = len(state)
print(f"n_observations: {n_observations}, n_actions: {n_actions}")

policy_net = DQN(n_observations, n_actions).to(device)
target_net = DQN(n_observations, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(10000)


steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return the largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.

            # HERE IS WHERE WE EXTRACT Q VALUE
            #print("Greedy Action")
            #for action in range(n_actions):
                #print(f"Q-value for action {action}: {policy_net(state)[0][action]}")
            #print()
            return policy_net(state).max(1).indices.view(1, 1)
    else:
        #print("Random Action\n")
        return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)

n_observations: 2, n_actions: 4


In [8]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1).values
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

In [9]:
@torch.no_grad()
def generate_policy_quiver(policy_net, env, episode_num, save_dir=HEATMAP_DIR):
    """
    Draws an arrow at each grid cell pointing in the direction of the
    greedy (argmax-Q) action. Skips blocks and the pit; puts a dot on the goal.
    Assumes env observation is [row, col].
    """
    g = env.grid_size
    # Grid of cell centers
    cols = np.arange(g)
    rows = np.arange(g)
    X, Y = np.meshgrid(cols, rows)  # X ~ col, Y ~ row

    # U = x-component (columns), V = y-component (rows)
    U = np.zeros((g, g), dtype=float)
    V = np.zeros((g, g), dtype=float)

    # Mask out arrows on walls/pit/goal
    mask = np.zeros((g, g), dtype=bool)
    block_set = set(env.blocks)
    pit = tuple(env.pit)
    goal = tuple(env.goal_pos)

    for r in range(g):
        for c in range(g):
            if (r, c) in block_set or (r, c) == pit or (r, c) == goal:
                mask[r, c] = True
                continue

            s = torch.tensor([[float(r), float(c)]], device=device, dtype=torch.float32)
            a = int(policy_net(s).argmax(dim=1).item())

            # Your action convention: 0:Up, 1:Right, 2:Down, 3:Left
            if a == 0:      # Up    -> row-1
                U[r, c], V[r, c] = 0.0, -1.0
            elif a == 1:    # Right -> col+1
                U[r, c], V[r, c] = 1.0, 0.0
            elif a == 2:    # Down  -> row+1
                U[r, c], V[r, c] = 0.0, 1.0
            elif a == 3:    # Left  -> col-1
                U[r, c], V[r, c] = -1.0, 0.0

    # Mask arrays so quiver skips those cells
    U = np.ma.array(U, mask=mask)
    V = np.ma.array(V, mask=mask)

    fig, ax = plt.subplots()
    ax.set_aspect('equal')
    ax.set_xlim(-0.5, g - 0.5)
    ax.set_ylim(-0.5, g - 0.5)
    ax.set_xticks(np.arange(g))
    ax.set_yticks(np.arange(g))
    ax.grid(True, which='both', linestyle='-', linewidth=0.5, alpha=0.4)

    # Invert Y so increasing row goes downward visually (matches grid indexing)
    ax.invert_yaxis()

    # Draw blocks
    for (br, bc) in block_set:
        ax.add_patch(plt.Rectangle((bc - 0.5, br - 0.5), 1, 1,
                                   facecolor=(0.5, 0.5, 0.55), edgecolor='none', alpha=0.9))
    # Pit
    pr, pc = pit
    ax.add_patch(plt.Rectangle((pc - 0.5, pr - 0.5), 1, 1,
                               facecolor=(0.8, 0.25, 0.25), edgecolor='none', alpha=0.9))
    # Goal
    gr, gc = goal
    ax.add_patch(plt.Rectangle((gc - 0.5, gr - 0.5), 1, 1,
                               facecolor=(0.3, 0.75, 0.3), edgecolor='none', alpha=0.9))

    # Quiver: arrows centered in each cell
    q = ax.quiver(X, Y, U, V, angles='xy', scale_units='xy', scale=1.5, width=0.01)

    ax.set_title(f"Greedy Policy Arrows — Episode {episode_num}")
    out_path = os.path.join(save_dir, f"policy_quiver_ep{episode_num:04d}.png")
    fig.savefig(out_path, bbox_inches='tight')
    plt.close(fig)
    return out_path


ACTION_TO_STR = {
    0: "up",   # up
    1: "right",   # right
    2: "down",   # down
    3: "left",   # left
}

@torch.no_grad()
def visualize_action_values(policy_net, state, global_timestep):
    # please change the x axis to action string
    action_strings = [ACTION_TO_STR[i] for i in range(n_actions)]
    q_values = policy_net(state)
    q_values = q_values.cpu().numpy().flatten()
    plt.bar(action_strings, q_values)
    plt.xlabel('Action')
    plt.ylabel('Q-value')
    plt.title(f'Action-Value Estimates at Timestep {global_timestep} for State {state.cpu().numpy()[0]}')
    viz_path = os.path.join(OUT_DIR, "action_maps",f"action_map{global_timestep:04d}.png")
    plt.savefig(viz_path)
    plt.clf()
    return viz_path

def save_frame(frame, global_timestep):
    frame_path = os.path.join(OUT_DIR, "frames", f"frame_{global_timestep:04d}.png")
    imageio.imwrite(frame_path, frame)
    plt.close()
    return frame_path

def build_gif_from_frames(frame_paths, gif_path, fps=GIF_FPS, cleanup=True):
    frames = [imageio.imread(p) for p in frame_paths]
    imageio.mimsave(gif_path, frames, fps=fps)
    if cleanup:
        for p in frame_paths:
            os.remove(p)
    return gif_path
    


In [10]:
num_episodes = 200
global_step = 0  # counts actions taken

frame_paths = []
action_value_viz_paths = []

for i_episode in tqdm(range(num_episodes)):
    # Initialize the environment and get its state
    state, info = env.reset()
    episode_reward = 0
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    
    for t in count():
        state_for_qviz = state.clone()
        action = select_action(state)
        observation, reward, terminated, truncated, info = env.step(action.item())
        frame = env.render()
        episode_reward += reward
        reward = torch.tensor([reward], device=device)
        done = terminated or truncated

        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)
        
        global_step += 1

        pth = save_frame(frame, global_step)
        frame_paths.append(pth)
        
        pth = visualize_action_values(policy_net, state_for_qviz, global_step)
        action_value_viz_paths.append(pth)
        
        
        if done:
            generate_policy_quiver(policy_net, env, i_episode)
            build_gif_from_frames(
                frame_paths,
                os.path.join(GIF_DIR, f"episode_{i_episode:04d}.gif"),
                fps=GIF_FPS,
                cleanup=True
            )
            frame_paths = []  # reset for next episode
            
            build_gif_from_frames(
                action_value_viz_paths,
                os.path.join(GIF_DIR, f"action_values_ep{i_episode:04d}.gif"),
                fps=GIF_FPS,
                cleanup=True
            )
            action_value_viz_paths = []  # reset for next episode
            break

print('Complete')
plt.ioff()
plt.show()

  0%|          | 0/200 [00:00<?, ?it/s]

Complete


<Figure size 640x480 with 0 Axes>